# Tune GPT2 to generate positive reviews
> Optimise GPT2 to produce positive IMDB movie reviews using a BERT sentiment classifier as a reward function.

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2_bert_training.png' width='600'>
<p style="text-align: center;"> <b>Figure:</b> Experiment setup to tune GPT2. The yellow arrows are outside the scope of this notebook, but the trained models are available through Hugging Face. </p>
</div>


In this notebook we fine-tune GPT2 (small) to generate positive movie reviews based on the IMDB dataset. The model gets the start of a real review and is tasked to produce positive continuations. To reward positive continuations we use a BERT classifier to analyse the sentiment of the produced sentences and use the classifier's outputs as rewards signals for PPO training.

## Setup experiment

### Import dependencies

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install transformers trl wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7

In [ ]:
import torch
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

import torch.nn.functional as F

### Configuration

In [ ]:
batch_size =128
version_code = 'test-colab-roberta256-16-1_0'

config = PPOConfig(
    model_name="lvwerra/gpt2-imdb",
    # model_name='/content/drive/MyDrive/cs234-project/models/dpo_test_0_2-2024.05.22.22.34',
    # model_name="edbeeching/gpt2-large-imdb",
    learning_rate=1.41e-5,
    # log_with="wandb",
    # init_kl_coef=0.02,
    init_kl_coef=1.0,
    # target=10,
    adap_kl_ctrl=False,
    # steps=1000,
    batch_size=batch_size,
    mini_batch_size=batch_size,
    # reward_model="sentiment-analysis:lvwerra/distilbert-imdb",
    reward_model="sentiment-analysis:siebert/sentiment-roberta-large-english",
)

sent_kwargs = {"return_all_scores": True, "function_to_apply": "softmax", "batch_size": 32}

In [ ]:
import wandb

wandb.login(key="KEY")
wandb.init(name=version_code)

from huggingface_hub import notebook_login,login

# notebook_login("KEY")
login("KEY")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mjh388 (mhayes). Use `wandb login --relogin` to force relogin


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


You can see that we load a GPT2 model called `gpt2_imdb`. This model was additionally fine-tuned on the IMDB dataset for 1 epoch with the huggingface [script](https://github.com/huggingface/transformers/blob/master/examples/run_language_modeling.py) (no special settings). The other parameters are mostly taken from the original paper ["Fine-Tuning Language Models from Human Preferences"](
https://arxiv.org/pdf/1909.08593.pdf). This model as well as the BERT model is available in the Huggingface model zoo [here](https://huggingface.co/models). The following code should automatically download the models.

## Load data and models

### Load IMDB dataset
The IMDB dataset contains 50k movie review annotated with "positive"/"negative" feedback indicating the sentiment.  We load the IMDB dataset into a DataFrame and filter for comments that are at least 200 characters. Then we tokenize each text and cut it to random size with the `LengthSampler`.

In [ ]:
def build_dataset(config, dataset_name="imdb", input_min_text_length=2, input_max_text_length=8):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    # load imdb with datasets
    ds = load_dataset(dataset_name, split="train")
    ds = ds.rename_columns({"text": "review"})
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)

    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["review"])[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

In [ ]:
dataset = build_dataset(config)


def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/24895 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1168 > 1024). Running this sequence through the model will result in indexing errors


### Load pre-trained GPT2 language models

We load the GPT2 model with a value head and the tokenizer. We load the model twice; the first model is optimized while the second model serves as a reference to calculate the KL-divergence from the starting point. This serves as an additional reward signal in the PPO training to make sure the optimized model does not deviate too much from the original language model.

In [ ]:
# model = AutoModelForCausalLMWithValueHead.from_pretrained('DownwardSpiral33/gpt2-imdb-pos-roberta16-256_0_02-2024.05.20.07.52')
# model = AutoModelForCausalLMWithValueHead.from_pretrained('/content/drive/MyDrive/cs234-project/models/test_dpo_0_1_lr2e_6-2024.06.10.01.36/checkpoint-2218')
# model = AutoModelForCausalLMWithValueHead.from_pretrained('DownwardSpiral33/gpt2-imdb-pos-roberta16-256_0_03-normalize-2024.06.02.22.36')
# model = AutoModelForCausalLMWithValueHead.from_pretrained('DownwardSpiral33/gpt2-imdb-pos-roberta16-256_0_05-gamma-2024.06.06.07.59')
# model = AutoModelForCausalLMWithValueHead.from_pretrained('DownwardSpiral33/gpt2-imdb-pos-roberta16-256_0_2-2024.05.20.04.41') # 1.65
model = AutoModelForCausalLMWithValueHead.from_pretrained('DownwardSpiral33/gpt2-imdb-pos-roberta16-256_0_27-normalize-2024.06.04.09.30')

ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)

tokenizer.pad_token = tokenizer.eos_token

Some weights of the model checkpoint at DownwardSpiral33/gpt2-imdb-pos-roberta16-256_0_27-normalize-2024.06.04.09.30 were not used when initializing GPT2LMHeadModel: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Initialize PPOTrainer
The `PPOTrainer` takes care of device placement and optimization later on:

In [ ]:
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=dataset, data_collator=collator)

### Load BERT classifier
We load a BERT classifier fine-tuned on the IMDB dataset.

In [ ]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug
sentiment_pipe = pipeline(config.reward_model.split(':')[0], model=config.reward_model.split(':')[1], device=device)

The model outputs are the logits for the negative and positive class. We will use the logits for positive class as a reward signal for the language model.

In [ ]:
text = "this movie was really bad!!"
sentiment_pipe(text, **sent_kwargs)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'NEGATIVE', 'score': 0.9994950294494629},
  {'label': 'POSITIVE', 'score': 0.0005049362080171704}]]

In [ ]:
text = "this movie was really good!!"
sentiment_pipe(text, **sent_kwargs)

[[{'label': 'NEGATIVE', 'score': 0.0012812110362574458},
  {'label': 'POSITIVE', 'score': 0.9987188577651978}]]

### Generation settings
For the response generation we just use sampling and make sure top-k and nucleus sampling are turned off as well as a minimal length.

In [ ]:
# gen_kwargs = {"min_length": -1, "top_k": 0.0, "top_p": 1.0, "do_sample": True, "pad_token_id": tokenizer.eos_token_id}

## Optimize model

### Training loop

The training loop consists of the following main steps:
1. Get the query responses from the policy network (GPT-2)
2. Get sentiments for query/responses from BERT
3. Optimize policy with PPO using the (query, response, reward) triplet

**Training time**

This step takes **~2h** on a V100 GPU with the above specified settings.

In [ ]:
# # output_min_length = 4
# # output_max_length = 16
# # output_length_sampler = LengthSampler(output_min_length, output_max_length)

# print('dataloader len', len(ppo_trainer.dataloader))

# generation_kwargs = {
#     "min_length": -1,
#     "top_k": 0.0,
#     "top_p": 1.0,
#     "do_sample": True,
#     "pad_token_id": tokenizer.eos_token_id,
#     "max_new_tokens": 16,
# }

# # def sftmx(output):1
# #   return torch.exp(torch.tensor(output[1]["score"]))/(torch.exp(torch.tensor(output[1]["score"]))+torch.exp(torch.tensor(output[0]["score"])))


# for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
#     # if epoch > 0:
#     #     break
#     query_tensors = batch["input_ids"]

#     #### Get response from gpt2
#     response_tensors = []
#     # for query in query_tensors:
#     #     # gen_len = output_length_sampler()
#     #     # generation_kwargs["max_new_tokens"] = gen_len
#     #     response = ppo_trainer.generate(query, **generation_kwargs)
#     #     response_tensors.append(response.squeeze()[-gen_len:])
#     response_tensors, ref_response_tensors = ppo_trainer.generate(
#         query_tensors, return_prompt=False, generate_ref_response=True, **generation_kwargs
#     )
#     batch["response"] = tokenizer.batch_decode(response_tensors)
#     batch["ref_response"] = tokenizer.batch_decode(ref_response_tensors)

#     #### Compute sentiment score
#     texts = [q + r for q, r in zip(batch["query"], batch["response"])]
#     pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
#     rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]
#     # rewards = [sftmx(output) for output in pipe_outputs]

#     #### Run PPO step
#     stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
#     ppo_trainer.log_stats(stats, batch, rewards)

### Training progress
If you are tracking the training progress with Weights&Biases you should see a plot similar to the one below. Check out the interactive sample report on wandb.ai: [link](https://app.wandb.ai/huggingface/trl-showcase/runs/1jtvxb1m/).

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2_tuning_progress.png' width='800'>
<p style="text-align: center;"> <b>Figure:</b> Reward mean and distribution evolution during training. </p>
</div>

One can observe how the model starts to generate more positive outputs after a few optimisation steps.

> Note: Investigating the KL-divergence will probably show that at this point the model has not converged to the target KL-divergence, yet. To get there would require longer training or starting with a higher initial coefficient.

## Model inspection
Let's inspect some examples from the IMDB dataset. We can use `model_ref` to compare the tuned model `model` against the model before optimisation.

In [ ]:
type(model)

trl.models.modeling_value_head.AutoModelForCausalLMWithValueHead

In [ ]:
# df_batch = [df_batch[-3]]

In [ ]:
# type(df_batch[0])

In [ ]:
from datasets import load_dataset, concatenate_datasets, Dataset
def build_dataset(config, dataset_name="imdb", input_min_text_length=2, input_max_text_length=8):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    # load imdb with datasets
    # ds = load_dataset(dataset_name, split="train")
    ds = Dataset.from_pandas(pd.DataFrame({
        'review': [
            'My interest was raised as',
            'Rated R for Strong Language,',
            'Jörg',
            "It's bad."
        ]
    }))
    # ds = ds.rename_columns({"text": "review"})
    # ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)

    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["review"])#[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds
ds = build_dataset(config)

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [ ]:
#### get a batch from the dataset
bs = len(ds)
game_data = dict()
dataset.set_format("pandas")
# df_batch = dataset[:].sample(bs)
game_data["query"] = ds["query"]
query_tensors = ds["input_ids"]
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 16,
}

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    # gen_len = output_length_sampler()
    output = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), **generation_kwargs
    ).squeeze()#[-gen_len:]
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), **generation_kwargs
    ).squeeze()#[-gen_len:]
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [torch.tensor(output[1]["score"]) for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [torch.tensor(output[1]["score"])for output in sentiment_pipe(texts, **sent_kwargs)]

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

<ipython-input-185-1270d8308c34>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), **generation_kwargs
<ipython-input-185-1270d8308c34>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), **generation_kwargs
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


,query,response (before),response (after),rewards (before),rewards (after)
0,My interest was raised as,My interest was raised as it was one of the fe...,My interest was raised as a young man. I want ...,tensor(0.9978),tensor(0.9978)
1,"Rated R for Strong Language,","Rated R for Strong Language, Concluding commen...","Rated R for Strong Language, Strong Editing, R...",tensor(0.9988),tensor(0.9980)
2,Jörg,Jörg makes this movie immediately recognizable...,"Jörg, who was Danish actress who played a prof...",tensor(0.9987),tensor(0.9954)
3,It's bad.,It's bad. Not the broad-spectrum Tiny Gun and ...,"It's bad. If you can't watch it well enough, y...",tensor(0.0005),tensor(0.0006)


In [ ]:
df_results

,query,response (before),response (after),rewards (before),rewards (after)
0,My interest was raised as,My interest was raised as one can't really fol...,My interest was raised as I fell in love with ...,tensor(0.9939),tensor(0.9989)
1,"Rated R for Strong Language,","Rated R for Strong Language, This film is unim...","Rated R for Strong Language, Drama, Romance, P...",tensor(0.0005),tensor(0.9959)
2,Jörg,Jörg Zborowski<br /><br />Hard to believe Shab...,Jörg Forbrichn - Die Därftunden - Total Irene,tensor(0.0016),tensor(0.9949)
3,It's bad.,It's bad. It's bad for the women at which the ...,It's bad. Why not stop making movies? Beckett ...,tensor(0.0005),tensor(0.0009)


In [ ]:
# model.generate(
#         torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs)

Looking at the reward mean/median of the generated sequences we observe a significant difference.

In [ ]:
print("mean:")
display(df_results[["rewards (before)", "rewards (after)"]].mean())
print()
print("median:")
display(df_results[["rewards (before)", "rewards (after)"]].median())

mean:


rewards (before)    0.249137
rewards (after)     0.747627
dtype: object


median:


rewards (before)    0.001073
rewards (after)     0.995385
dtype: object

## Save model
Finally, we save the model and push it to the Hugging Face for later usage.

In [ ]:
model.save_pretrained(f"gpt2-imdb-pos-v2-{version_code}", push_to_hub=True)
tokenizer.save_pretrained(f"gpt2-imdb-pos-v2-{version_code}", push_to_hub=True)

('gpt2-imdb-pos-v2-test-colab-roberta256-16-1_0/tokenizer_config.json',
 'gpt2-imdb-pos-v2-test-colab-roberta256-16-1_0/special_tokens_map.json',
 'gpt2-imdb-pos-v2-test-colab-roberta256-16-1_0/vocab.json',
 'gpt2-imdb-pos-v2-test-colab-roberta256-16-1_0/merges.txt',
 'gpt2-imdb-pos-v2-test-colab-roberta256-16-1_0/added_tokens.json',
 'gpt2-imdb-pos-v2-test-colab-roberta256-16-1_0/tokenizer.json')

In [ ]:
!pip install trl
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead

model = AutoModelForCausalLMWithValueHead.from_pretrained("DownwardSpiral33/gpt2-imdb-pos-v2-roberta128-0_1")

Some weights of the model checkpoint at DownwardSpiral33/gpt2-imdb-pos-v2-roberta128-0_1 were not used when initializing GPT2LMHeadModel: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
test_dataset = build_dataset(config, 'test')

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [ ]:
# ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
dataloader = ppo_trainer.prepare_dataloader(test_dataset, collator)
with torch.no_grad():
  #### get a batch from the dataset
  bs = 2
  game_data = dict()
  # dataset.set_format("pandas")
  # df_batch = dataset[:].sample(bs)
  # game_data["query"] = df_batch["query"].tolist()
  # query_tensors = df_batch["input_ids"].tolist()
  device = 'cpu'

  response_tensors_ref, response_tensors = [], []

  generation_kwargs = {
      "min_length": -1,
      "top_k": 0.0,
      "top_p": 1.0,
      "do_sample": True,
      "pad_token_id": tokenizer.eos_token_id,
      "max_new_tokens": 16,
      # 'output_logits': True,
      # 'return_dict_in_generate': True,
  }

  for epoch, batch in tqdm(enumerate(dataloader)):
    query_tensors = batch["input_ids"]

    #### Get response from gpt2
    response_tensors = []
    # for query in query_tensors:
    #     # gen_len = output_length_sampler()
    #     # generation_kwargs["max_new_tokens"] = gen_len
    #     response = ppo_trainer.generate(query, **generation_kwargs)
    #     response_tensors.append(response.squeeze()[-gen_len:])
    response_tensors, ref_response_tensors = ppo_trainer.generate(
        query_tensors, return_prompt=False, generate_ref_response=True, **generation_kwargs
    )
    batch["response"] = tokenizer.batch_decode(response_tensors)
    batch["ref_response"] = tokenizer.batch_decode(ref_response_tensors)

    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]

    texts = [q + r for q, r in zip(batch["query"], batch["ref_response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    ref_rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]

    all_logprobs, logits_or_none, values, masks = ppo_trainer.batched_forward_pass(
        model,
        query_tensors,
        response_tensors,
        ppo_trainer.prepare_model_inputs(query_tensors, response_tensors),
        return_logits=True,
    )
    all_logprobs, logits_or_none, values, masks = ppo_trainer.batched_forward_pass(
        model,
        query_tensors,
        response_tensors,
        ppo_trainer.prepare_model_inputs(query_tensors, response_tensors),
        return_logits=True,
    )


  #### get response from gpt2 and gpt2_ref
  for i in range(bs):
      ppo_trainer.generate(
        query_tensors, return_prompt=False, generate_ref_response=True, **generation_kwargs
      )
      # gen_len = output_length_sampler()
      # output = ref_model.generate(
      #     torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), **generation_kwargs
      # )#[-gen_len:]
      # response_tensors_ref.append(output.sequences.squeeze())
      # output = model.generate(
      #     torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), **generation_kwargs
      # )#[-gen_len:]
      # response_tensors.append(output.sequences.squeeze())

      # kl = F.kl_div(log_probs_ref, log_probs, log_target=True, reduction='none')
      print(kl.shape)

  #### decode responses
  game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
  game_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

  #### sentiment analysis of query/response pairs before/after
  # texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
  # game_data["rewards (before)"] = [torch.tensor(output[1]["score"]) for output in sentiment_pipe(texts, **sent_kwargs)]

  texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
  # game_data["rewards (after)"] = [torch.tensor(output[1]["score"])for output in sentiment_pipe(texts, **sent_kwargs)]

  # store results in a dataframe
  df_results = pd.DataFrame(game_data)
df_results

0it [00:00, ?it/s]
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


NameError: name 'kl' is not defined